In [12]:
!pip install transformers==4.4.0

  Using cached transformers-4.4.0-py3-none-any.whl (2.1 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
ERROR: ratsnlp 1.0.5 has requirement transformers==4.10.0, but you'll have transformers 4.4.0 which is incompatible.
ERROR: kobert 0.2.3 has requirement transformers>=4.8.1, but you'll have transformers 4.4.0 which is incompatible.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0


In [1]:
import torch
import pandas as pd
import numpy as np

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from keras.preprocessing.sequence import pad_sequences

import time
# start = time.time()  # 시작 시간 저장
 
 
# # 작업 코드
 
 
# print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


Using TensorFlow backend.


In [12]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [13]:
# tokenizer import 
tokenizer = BertTokenizer.from_pretrained('klue/bert-base', do_lower_case=False)

In [14]:
# bert model 불러온 후 저장된 모델 상태 적용
model2 =  BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=5)
# model2.load_state_dict(torch.load('bert_model_ver2_st_0505.pt'))
model2.load_state_dict(torch.load('bert_model_ver2(77)_st_0505.pt'))

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

<All keys matched successfully>

In [15]:
# device 적용
device = torch.device("cuda:0")
model2.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [17]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model2.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model2(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits


In [ ]:
# sentences = pd.read_csv('review_test_data_total_ver2.csv', encoding = 'utf-8')
sentences_re = []
while True:
    i = 0
    start = time.time()  # 시작 시간 저장
    for review in sentences['review']:
        if i >2000:
            break
        print('리뷰를 입력해 주세요. : ')
        review = input()
        print('---------------------')
        logits = test_sentences([review])
#         print(review)
#         print(logits)
#         print(np.argmax(logits))
        if np.argmax(logits) == 0 :
#             print(review)
            print("⭐")
            print('-----------------------------------------------------------------------------------------------')
            sentences_re.append(1)
        elif np.argmax(logits) == 1 :
#             print(i)
            print("⭐⭐")
            print('-----------------------------------------------------------------------------------------------')
            sentences_re.append(2)
        elif np.argmax(logits) == 2 :
#             print(i)
            print("⭐⭐⭐")
            print('-----------------------------------------------------------------------------------------------')
            sentences_re.append(3)
        elif np.argmax(logits) == 3 :
#             print(i)
            print("⭐⭐⭐⭐")
            print('-----------------------------------------------------------------------------------------------')
            sentences_re.append(4)
        else :
#             print(i)
            print("⭐⭐⭐⭐⭐")
            print('-----------------------------------------------------------------------------------------------')
            sentences_re.append(5)
        i+=1

#     print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

리뷰를 입력해 주세요. : 
저렴한 가격으로 잘 샀어요 무난하게 사용하기 좋네요
---------------------
⭐⭐⭐⭐
-----------------------------------------------------------------------------------------------
리뷰를 입력해 주세요. : 
매트리스가 견고하고 탄탄하며 좋네요
---------------------
⭐⭐⭐⭐⭐
-----------------------------------------------------------------------------------------------
리뷰를 입력해 주세요. : 
침대 배송도 빠르고 친절하게 잘 설치해 주셔서 잘 사용하고 있습니다 수납공간도 넓어서 좋아요
---------------------
⭐⭐⭐⭐⭐
-----------------------------------------------------------------------------------------------
리뷰를 입력해 주세요. : 
사무실에서 피곤할 때 사용하려고 샀는데 실밥이 너무 많이 나와 있어요 ㅠㅠ
---------------------
⭐⭐
-----------------------------------------------------------------------------------------------
리뷰를 입력해 주세요. : 
위에서 움직일 때 흔들림이 있어요
---------------------
⭐⭐
-----------------------------------------------------------------------------------------------
리뷰를 입력해 주세요. : 
높이가 약간 낮은 감이 있어서 조금 아쉬워요
---------------------
⭐⭐
-------------------------------------------------------------------------------